In [28]:
import scanpy as sc
#import squidpy as sq
import os

In [29]:
# import sys
# sys.path.append('/home/augusta/SSS_mount/insituCNV/InSituCNV/modules')
# sys.path

# Functions

In [30]:
import random
import pandas as pd
import numpy as np
import anndata

## generate_cnvs

In [31]:
def generate_cnvs(CNV_dict, min_size, max_size, gene_info, save_csv = None):
    """ 
    This function generates copy number variations (CNVs) based on a dictionary (CNV_dict) of genes (keys) and whether they should be gain or loss (value). The size (bp) of the CNV is a randomly chosen between min_size and max_size. The gene will be in the center of the CNV, so half of the size (bp) is subtracted (start) and added (end) from the center of the gene position ('Gene end (bp)'- 'Gene start (bp)')/2), according to the gene_list. The function returns the CNV information as a DataFrame, specifying the gene name, chromosome, size (bp), type (gain/loss), start (bp), end (bp).

    Parameters:
        - CNV_dict (dictionary): Gene name (key) and whether they should be 'gain' or 'loss'.
        - min_size (nbr): minimum size (in bp) of the CNV.
        - max_size (nbr): maximum size (in bp) of the CNV.
        - gene_info (str): The path to the gene information from the *Ensmbl_BioMart_gene_info.txt* file, including  'Gene stable ID', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)', 'Gene name' of the human genome GRCh38.

    Returns:
        - cnv_df (DataFrame): Compiling the gene name, chromosome, size, type, start, end
    """

    gene_info = pd.read_csv(gene_info)
    cnv_list = []

    for gene in CNV_dict:
        # Check if the gene is present in gene_info DataFrame
        if gene not in gene_info['Gene name'].values:
            print(f"Gene '{gene}' not found in gene_info DataFrame.")
            continue
        
        # Find gene details in the gene_info DataFrame
        gene_row = gene_info.loc[gene_info['Gene name'] == gene].iloc[0]
        
        # Calculate gene center
        gene_center = (gene_row['Gene end (bp)'] + gene_row['Gene start (bp)']) // 2
        
        # Randomly generate CNV size
        cnv_size = random.randint(min_size, max_size)
        
        # Calculate CNV start and end positions
        cnv_start = gene_center - cnv_size // 2
        cnv_end = gene_center + cnv_size // 2
        
        # Create CNV entry
        cnv = {
            'Gene name': gene,
            'Chromosome': gene_row['Chromosome/scaffold name'],
            'Size (bp)': cnv_size,
            'Type': CNV_dict[gene],
            'Start (bp)': cnv_start,
            'End (bp)': cnv_end
        }
        
        cnv_list.append(cnv)
    
    # Convert the list of CNV dictionaries to a DataFrame
    cnv_df = pd.DataFrame(cnv_list)

    if save_csv:
        cnv_df.to_csv(save_csv, index=None)
    
    return cnv_df

## create_cnv_template

In [32]:
import numpy as np
import pandas as pd

def create_cnv_template(adata, CNV_df):
    """ 
    This function generates a template for CNVs to an AnnData object.

    Parameters:
        - adata (AnnData): The AnnData object to which the CNVs should fit.
        - CNV_df (DataFrame): CNVs generated by the generate_cnvs function. 

    Returns:
        - cnv_template_df (DataFrame): A CNV template where each CNV (gene name) has a -1/0/1 value for each gene in the adata.var.
    """
    
    # Step 1: Order adata.var by chromosome and start position
    adata.var.sort_values(by=['chromosome', 'start'], inplace=True)
    
    # Initialize the CNV template matrix with zeros
    cnv_template = np.zeros((len(CNV_df), len(adata.var_names)))

    # Create a mapping of gene names to indices in the sorted AnnData object
    gene_to_index = {gene: idx for idx, gene in enumerate(adata.var_names)}

    # Step 2: Loop through each CNV in the CNV_df
    for i, row in CNV_df.iterrows():
        chromosome = row['Chromosome']
        cnv_start = row['Start (bp)']
        cnv_end = row['End (bp)']
        cnv_type = row['Type']
        
        # Find the genes in the corresponding chromosome that fall within the CNV region
        selected_genes = adata.var[
            (adata.var['chromosome'] == f"chr{chromosome}") &
            (adata.var['start'] >= cnv_start) &
            (adata.var['start'] <= cnv_end)
        ]
        
        # Determine the CNV effect: -1 for loss, +1 for gain
        cnv_effect = -1 if cnv_type == 'loss' else 1
        
        # Step 3: Set the corresponding entries in the CNV template matrix
        for gene in selected_genes.index:
            gene_index = gene_to_index[gene]
            cnv_template[i, gene_index] = cnv_effect
    
    # Convert the CNV template matrix to a DataFrame for easier interpretation
    cnv_template_df = pd.DataFrame(cnv_template, columns=adata.var_names)
    
    # Optionally, add the name of the CNV (gene name) as rownames in the DataFrame for reference
    if 'Gene name' in CNV_df.columns:
        cnv_template_df.index = CNV_df['Gene name'].values
    else:
        print("Warning: 'Gene name' column missing from CNV_df.")

    return cnv_template_df


## simulate_cnvs

In [33]:
import numpy as np
import pandas as pd

def simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference, cell_type_cnv, alpha=2):
    """ 
    This function creates a layer of the adata.X with simulated gains and losses according to the subclone dictionary.
    The cells are randomly distributed between subclones, with amplification or deletion determined by a uniformly drawn random value.

    Parameters:
        - adata (AnnData): The AnnData object where the CNVs should be simulated.
        - cnv_template_df (DataFrame): Templates for each CNV generated for the adata.
        - subclone_dict (dictionary): Specifies the subclones and their CNVs.
        - cell_type_reference (str): Column name in adata.obs indicating cell types.
        - cell_type_cnv (str or list): Specific cell types in which CNVs should be simulated.
        - alpha (float): The maximum amplitude for CNV change (default = 2).

    Returns:
        - adata (AnnData): The simulated AnnData object containing adata.obs['simulated_subclone'] 
                           and the simulated count matrix in an 'CNV_simulated' layer.
    """
    
    # Step 1: Create a new layer in adata to store the simulated CNV data
    adata.layers['CNV_simulated'] = adata.layers['counts'].copy()
    adata.layers['CNV_GT'] = np.zeros(adata.X.shape)

    # Step 2: Randomly assign cells to subclones based on subclone_dict
    if isinstance(cell_type_cnv, str):
        cell_type_cnv = [cell_type_cnv]  # Convert string to a list
    
    num_cells = adata[adata.obs[cell_type_reference].isin(cell_type_cnv)].shape[0]
    subclone_names = list(subclone_dict.keys())

    # Assign random subclone to cells of the desired cell type
    adata.obs.loc[adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = np.random.choice(subclone_names, size=num_cells, replace=True)
    # Assign 'N' to cells that are not in the selected cell type
    adata.obs.loc[~adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = "N"

    # Step 3: Apply CNVs to the expression data based on the assigned subclone
    for subclone, cnvs in subclone_dict.items():
        if not cnvs:
            # Skip if the CNV list is empty (as for normal subclone)
            continue
        
        # Get cells assigned to the current subclone
        subclone_cells = adata.obs['simulated_subclone'] == subclone
        
        for cnv in cnvs:
            # Extract the CNV effect for this gene from cnv_template_df
            cnv_row = cnv_template_df[cnv_template_df.index == cnv]
            cnv_effects = cnv_row.values.flatten()
            
            for gene_idx, effect in enumerate(cnv_effects):
                if effect == -1:
                    # Simulate deletion by drawing a random value rho in (0, alpha)
                    rho = np.random.uniform(0, alpha)
                    # Decrease counts by dividing by (1 + rho)
                    adata.layers['CNV_simulated'][subclone_cells, gene_idx] /= (1 + rho)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = -1
                elif effect == 1:
                    # Simulate amplification by drawing a random value rho in (0, alpha)
                    rho = np.random.uniform(0, alpha)
                    # Increase counts by multiplying by (1 + rho)
                    adata.layers['CNV_simulated'][subclone_cells, gene_idx] *= (1 + rho)
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = 1

    return adata

In [34]:
import numpy as np

def simulate_cnvs(adata, cnv_template_df, subclone_dict, cell_type_reference, cell_type_cnv, alpha):
    
    """ 
    This function creates a layer of the adata.X with simulated gains and losses according to the subclone dictionary.
    The cells are randomly distributed between subclones, with amplification or deletion determined by a uniformly drawn random value.

    Parameters:
        - adata (AnnData): The AnnData object where the CNVs should be simulated.
        - cnv_template_df (DataFrame): Templates for each CNV generated for the adata.
        - subclone_dict (dictionary): Specifies the subclones and their CNVs.
        - cell_type_reference (str): Column name in adata.obs indicating cell types.
        - cell_type_cnv (str or list): Specific cell types in which CNVs should be simulated.
        - alpha (float): The maximum amplitude for CNV change (default = 2).

    Returns:
        - adata (AnnData): The simulated AnnData object containing adata.obs['simulated_subclone'] 
                           and the simulated count matrix in an 'CNV_simulated' layer.
    """

    # Step 1: Create a new layer in adata to store the simulated CNV data
    adata.layers['CNV_simulated'] = adata.layers['counts'].copy()
    adata.layers['CNV_GT'] = np.zeros(adata.X.shape)

    # Step 2: Randomly assign cells to subclones based on subclone_dict
    if isinstance(cell_type_cnv, str):
        cell_type_cnv = [cell_type_cnv]  # Convert string to a list

    num_cells = adata[adata.obs[cell_type_reference].isin(cell_type_cnv)].shape[0]
    subclone_names = list(subclone_dict.keys())

    # Assign random subclone to cells of the desired cell type
    adata.obs.loc[adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = np.random.choice(subclone_names, size=num_cells, replace=True)
    # Assign 'N' to cells that are not in the selected cell type
    adata.obs.loc[~adata.obs[cell_type_reference].isin(cell_type_cnv), 'simulated_subclone'] = "N"

    # Step 3: Apply CNVs to the expression data based on the assigned subclone
    for subclone, cnvs in subclone_dict.items():
        if not cnvs:
            continue
        
        # Get cells assigned to the current subclone
        subclone_cells = adata.obs['simulated_subclone'] == subclone
        
        for cnv in cnvs:
            cnv_row = cnv_template_df[cnv_template_df.index == cnv]
            cnv_effects = cnv_row.values.flatten()
            
            for gene_idx, effect in enumerate(cnv_effects):
                if effect == -1:
                    # Simulate deletion
                    rho = np.random.uniform(0, alpha)
                    values_to_modify = adata.layers['CNV_simulated'][subclone_cells, gene_idx].todense().A1  # Flatten dense array
                    modified_values = values_to_modify / (1 + rho)
                    adata.layers['CNV_simulated'][subclone_cells, gene_idx] = modified_values[:, None]  # Reshape for compatibility
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = -1
                elif effect == 1:
                    # Simulate amplification
                    rho = np.random.uniform(0, alpha)
                    values_to_modify = adata.layers['CNV_simulated'][subclone_cells, gene_idx].todense().A1  # Flatten dense array
                    modified_values = values_to_modify * (1 + rho)
                    adata.layers['CNV_simulated'][subclone_cells, gene_idx] = modified_values[:, None]  # Reshape for compatibility
                    adata.layers['CNV_GT'][subclone_cells, gene_idx] = 1

    return adata


# Load AnnData

In [35]:
import scanpy as sc
adata_path = "/home/sergio/Jnotebooks/inferCNV/data/vascular_normal.h5ad"
adata = sc.read_h5ad(adata_path)

# Illustrate the genome coverage of the data

In [36]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

def plot_gene_coverage(adata, gene_identifyer, gene_info, count_max = 200000, save_html=None):
    """
    This function generates an interactive gene coverage plot for the Human Lymph Node based on gene information and an AnnData object.

    Parameters:
    - adata (AnnData): An AnnData object containing gene IDs and total counts in `adata.var`.
    - gene_identifyer (str): The variable name where the adata.var gene names are located. 
    - gene_info (str): The path to the gene information from the *Ensmbl_BioMart_gene_info.txt* file, including  'Gene stable ID', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)', 'Gene name' of the human genome GRCh38.
    - count_max (nbr): The maximum number of counts to plot in the y-axis.
    - save_html (str): The file path to save the plot as an HTML file. 

    Returns:
    - plot
    """

    # Load and preprocess gene data
    gene_data = pd.read_csv(gene_info, sep=',', header=0, names=['Gene ID', 'Chromosome', 'Start', 'End', 'Gene name'])
    gene_data_sorted = gene_data.sort_values(by=['Chromosome', 'Start'])

    # Initialize a matrix with zeros
    matrix = pd.DataFrame(0, index=gene_data_sorted['Gene ID'], columns=['data'])

    
    # Convert adata.var to a DataFrame for merging
     
    var_data = pd.DataFrame({
        'Gene ID': adata.var[gene_identifyer],
        'Total Counts': adata.var['total_counts']
    })

    # Merge gene data with total counts
    merged_data = gene_data_sorted.merge(var_data, on='Gene ID', how='left')
    merged_data.index = merged_data['Gene ID']

    # Fill the matrix with total counts
    matrix.loc[merged_data['Gene ID'], 'data'] = merged_data['Total Counts']

    # Create a column for combined chromosome and gene name
    gene_data_sorted['chrom_gene'] = gene_data_sorted['Chromosome'].astype(str) + ':' + gene_data_sorted['Gene name']

    # Create the Plotly bar plot
    fig = px.bar(
        matrix, 
        x=gene_data_sorted['chrom_gene'], 
        y='data', 
        title='Gene Coverage',
        labels={'x': 'Gene name', 'data': 'Total Counts'},
        color_discrete_sequence=['blue']
    )

    # Update layout for better visualization
    fig.update_layout(
        yaxis_range=[0, count_max],  # Set y-axis limit
        width=1200,  # Set plot width
        height=400,  # Set plot height
        plot_bgcolor='white', 
        paper_bgcolor='white'
    )


    # Save the plot as an HTML file
    if save_html:
        pio.write_html(fig, file=save_html, auto_open=True)

    return fig

In [37]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

def plot_gene_coverage(adata, gene_identifier, gene_info, count_max=200000, save_html=None):
    """
    This function generates an interactive gene coverage plot for the Human Lymph Node based on gene information and an AnnData object.

    Parameters:
    - adata (AnnData): An AnnData object containing gene IDs and total counts in `adata.var`.
    - gene_identifier (str): The variable name where the `adata.var` gene names are located. 
    - gene_info (str): The path to the gene information from the *Ensmbl_BioMart_gene_info.txt* file, which should include 'Gene stable ID', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)', 'Gene name' of the human genome GRCh38.
    - count_max (int): The maximum number of counts to plot on the y-axis.
    - save_html (str): The file path to save the plot as an HTML file. 

    Returns:
    - fig: The Plotly figure object.
    """
    # Load and preprocess gene data
    gene_data = pd.read_csv(gene_info, sep=',', header=0, names=['Gene ID', 'Chromosome', 'Start', 'End', 'Gene name'])
    gene_data_sorted = gene_data.sort_values(by=['Chromosome', 'Start'])

    # Extract gene count data from adata.var and merge it with gene_data
    var_data = pd.DataFrame({
        'Gene ID': adata.var[gene_identifier],
        'Total Counts': adata.var['total_counts']
    }).reset_index(drop=True)  # Ensure index alignment if needed

    # Merge gene data with total counts from adata.var
    merged_data = gene_data_sorted.merge(var_data, on='Gene ID', how='left')
    merged_data['Total Counts'].fillna(0, inplace=True)  # Fill missing values with 0

    # Add combined chromosome and gene name for x-axis labels
    merged_data['chrom_gene'] = merged_data['Chromosome'].astype(str) + ':' + merged_data['Gene name']

    # Create the Plotly bar plot
    fig = px.bar(
        merged_data,
        x='chrom_gene', 
        y='Total Counts', 
        title='Gene Coverage',
        labels={'chrom_gene': 'Gene (Chromosome:Gene Name)', 'Total Counts': 'Total Counts'},
        color_discrete_sequence=['blue']
    )

    # Update layout for better visualization
    fig.update_layout(
        yaxis_range=[0, count_max],  # Set y-axis limit
        width=1200,  # Set plot width
        height=400,  # Set plot height
        plot_bgcolor='white', 
        paper_bgcolor='white'
    )

    # Save the plot as an HTML file
    if save_html:
        pio.write_html(fig, file=save_html, auto_open=True)

    return fig


In [38]:
gene_file = os.path.expanduser("/home/sergio/Jnotebooks/inferCNV/InSituCNV/Ensmbl_BioMart_gene_info.txt")

In [39]:
#plot_gene_coverage(adata, count_max=400000, gene_identifier= 'feature_name', gene_info=gene_file, save_html='test.html')

# Simulate the data to contain CNVs

Divide the dataset into four simulated subclones

1.   Normal (unaltered)
2.   Subclone A (fewer CNVs)
3.   Subclone B (same as A but added)
4.   Subclone C (same as A but added)


Choose chromosomal regions to be duplicated or deleted

If we generate several CNVs throughout the genome, it might be possible to compare the detection efficiency as to how many of the CNVs are detected. Or else, it's works/not works.


Things to consider: 
- how the size (nbr of cells) of the subclone affect the outcome - or make same size populations and avoid adressing this
- choose the size of the CNVs
    - literature to find an appropriate size for CNVs (50bp - several Mbs, ref: https://doi.org/10.1016%2Fj.bj.2021.02.003)
    - different sizes to see how that affects the outcome - or make every CNV the same size


## Module to simulate CNVs in adata (simulate_CNVs.py)

In [42]:
import sys
sys.path.append("/home/sergio/Jnotebooks/inferCNV/InSituCNV/modules")
from simulate_CNVs import *

### Function: Generate CNVs (generate_cnvs)

This function generates copy number variations (CNVs) based on a dictionary (**CNV_dict**) of genes (keys) and whether they should be gain or loss (value). The size (bp) of the CNV is a randomly chosen size between **min_size** and **max_size**. The gene will be in the center of the CNV, so half of the size (bp) is subtracted (start) and added (end) from the center of the gene position ('Gene end (bp)'- 'Gene start (bp)')/2), taken from the **gene_list**. The output will be a list of these CNVs, specifying the gene name, chromosome, size (bp), type (gain/loss), start (bp), end (bp).

Parameters:

- **CNV_dict** - dict. Gene name (key) and whether they should be 'gain' or 'loss'. Could for example be gain of known oncogenes or loss of tumor supressors. Gene selection inspo: https://doi.org/10.1080%2F07853890.2023.2280708
- **min_size** - nbr in bp.
- **max_size** - nbr in bp. A size of the CNV is generated as a random number between the min and max
- **gene_info** - the *Ensmbl_BioMart_gene_info.txt* containing 'Gene stable ID', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)', 'Gene name'

Returns:

- **CNV_df** (DataFrame): Compiling the gene name, chromosome, size, type, start, end

In [43]:
def gene_exists(adata, gene_list):
    for gene in gene_list:
        if gene in adata.var_names:
            print(gene, 'exists! \n')
        else:
            print(gene, 'does not exist.. \n')

In [47]:
adata.var.index=adata.var['feature_name']

In [48]:
gene_exists(adata, gene_list=['DIS3','MECOM','ERBB2','CHD7','HCK','KEAP1','MYD88','TBX3']) #'EPHB1','FLT1',

# CNVs, including ABCC5, AGO2, ARID5B, CHD7, FAM58A, FOXA1, HEY1, HLA-C, HLA-DQB1, MCL1, MECOM, MSN, NFKBIA, PRSS1, RAD21,

DIS3 exists! 

MECOM exists! 

ERBB2 exists! 

CHD7 exists! 

HCK exists! 

KEAP1 exists! 

MYD88 exists! 

TBX3 exists! 



In [49]:
CNV_dict = { 
    'DIS3': 'loss',
    'MECOM': 'loss',
    'ERBB2': 'gain',
    'CHD7': 'gain',
    'HCK': 'gain',
    'KEAP1': 'loss',
    'MYD88': 'gain',
    'TBX3': 'gain'
}

min_size = 10000
max_size = 1000000


gene_file = ("/home/sergio/Jnotebooks/inferCNV/InSituCNV/Ensmbl_BioMart_gene_info.txt")

In [50]:
CNV_df = generate_cnvs(CNV_dict, min_size, max_size, gene_info=gene_file, save_csv='CNVs_301024.csv') 
CNV_df

,Gene name,Chromosome,Size (bp),Type,Start (bp),End (bp)
0,DIS3,13,773161,loss,72380552,73153712
1,MECOM,3,128588,loss,169309343,169437931
2,ERBB2,17,733637,gain,39342352,40075988
3,CHD7,8,55658,gain,60745555,60801213
4,HCK,20,290236,gain,31931908,32222144
5,KEAP1,19,353939,loss,10317872,10671810
6,MYD88,3,409588,gain,37935994,38345582
7,TBX3,12,631883,gain,114361274,114993156


#### ...or extract CNVs

In [28]:
CNV_df = pd.read_csv("/home/augusta/SSS_mount/insituCNV/InSituCNV/Figure2/01_Simulate_CNVs_in_spatial_data/Simulate_CNVs/CNVs_301024.csv")

In [29]:
CNV_df

,Gene name,Chromosome,Size (bp),Type,Start (bp),End (bp)
0,DIS3,13,562238,loss,72486013,73048251
1,MECOM,3,15767,loss,169365754,169381520
2,ERBB2,17,803723,gain,39307309,40111031
3,CHD7,8,523984,gain,60511392,61035376
4,HCK,20,954807,gain,31599623,32554429
5,KEAP1,19,875978,loss,10056852,10932830
6,MYD88,3,865647,gain,37707965,38573611
7,TBX3,12,11895,gain,114671268,114683162


### Function: Create CNV template (create_cnv_template)

This function generates a CNV template for an AnnData object based on a predifined number of subclones, and their CNVs, generated by the generate_cnvs function.  

Parameters:

- **adata** - the AnnData object
- **CNV_df** (DataFrame): from the *'Generate CNVs'* module. the gene name, chromosome, size, type, start, end - . The type should generate either -1/+1 depending on if it's 'gain' or 'loss'.

Returns:

- **cnv_template_df** (DataFrame). For each CNV (gene name) - generate an array with -1/0/1 values for each gene in the adata.var_names

In [51]:
cnv_template_df = create_cnv_template(adata, CNV_df)
cnv_template_df

/home/sergio/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:750: UserWarning:


AnnData expects .var.index to contain strings, but got values like:
    ['ENSG00000238009.6', 'ENSG00000239945.1', 'LINC01409', 'FAM87B', 'LINC00115']

    Inferred to be: categorical




feature_name,ENSG00000238009.6,ENSG00000239945.1,LINC01409,FAM87B,LINC00115,FAM41C,LINC02593,SAMD11,NOC2L,KLHL17,...,ENSG00000277400.1,MAFIP,ENSG00000276256.1,ENSG00000273748.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278384.1,ENSG00000276345.1,ENSG00000271254.7
DIS3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MECOM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERBB2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHD7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KEAP1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MYD88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TBX3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Function: Simualte CNVs in the data (simulate_cnvs)

This function applies poission distribution / binomial probability to simulate one arm gains and losses (increase/keep/decrease the counts (adata.X) depending on the -1/0/1 value of the cnv_template_df: if -1, the probability is 0.5 and if 1 it is 1.5). This is done according to a subclone_dict, where each key is a subclone, and the values state which CNVs to add to these cells.  

Parameters:

- **adata** - the AnnData object to add the CNVs on
- **cnv_template_df** matrix. from the *'Create CNV template'* module
- **subclone_dict** (dictionary) For each subclone (key), a list of which CNVs to assign (values). randomly assigning each cell to one of these subclones (adata.obs.simulated_subclone)

Returns:

- **adata** - the AnnData object with simulated CNVs as a layer to the adata.X, (as well as the CNV template as a 'CNV_template' layer?)

In [52]:
adata

AnnData object with n_obs × n_vars = 6184 × 22798
    obs: 'mapped_reference_annotation', 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'donor_living_at_sample_collection', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'sample_derivation_process', 'donor_BMI_at_collection', 'tissue_type', 'suspension_derivation_process', 'suspension_dissociation_reagent', 'suspension_dissociation_time', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'sequencing_platform', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'sex_ontology_term_id', 'nCount_RNA', 'nFeature_RNA', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_

In [53]:
subclone_dict = {
    'N': [], # Normal cells without CNV simulations
    'A': ['DIS3', 'MECOM', 'ERBB2', 'CHD7'], # Original genetic subclone
    'B': ['DIS3', 'MECOM', 'ERBB2', 'CHD7', 'HCK', 'KEAP1'], 
    'C': ['DIS3', 'MECOM', 'ERBB2', 'CHD7', 'MYD88', 'TBX3'],
} # 'DIS3','MECOM','ERBB2','CHD7','HCK','KEAP1','MYD88','TBX3'

In [56]:
simulate_cnvs?

In [58]:
simulate_cnvs(adata, cnv_template_df, subclone_dict)# cell_type_reference='cell_type',cell_type_cnv='fibroblast', alpha=6)

AnnData object with n_obs × n_vars = 6184 × 22798
    obs: 'mapped_reference_annotation', 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'donor_living_at_sample_collection', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'sample_derivation_process', 'donor_BMI_at_collection', 'tissue_type', 'suspension_derivation_process', 'suspension_dissociation_reagent', 'suspension_dissociation_time', 'suspension_uuid', 'suspension_type', 'library_uuid', 'assay_ontology_term_id', 'sequencing_platform', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'sex_ontology_term_id', 'nCount_RNA', 'nFeature_RNA', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_

In [59]:
adata.obs.cell_type

AAACGAAAGACTCATC-1    macrophage
AAACGAACAACGACTT-1    macrophage
AAACGCTCAGCAGGAT-1    fibroblast
AAACGCTTCCGTGTCT-1    macrophage
AAAGGATTCCAACACA-1    macrophage
                         ...    
TTTGGTTCAACAGATA-9    macrophage
TTTGTTGAGGCGTCCT-9    fibroblast
TTTGTTGTCGAACTCA-9    macrophage
TTTGTTGTCGTTAGTG-9    fibroblast
TTTGTTGTCTTGCAAG-9    fibroblast
Name: cell_type, Length: 6184, dtype: category
Categories (2, object): ['fibroblast', 'macrophage']

In [45]:
adata.write("vascular_normal_simulatedCNVs_301024.h5ad", compression = 'gzip')